# Post-process vasca_pipe results
Make catalog file of selected sources only, associate and get store additional info for listed sources

In [1]:
region_name = "ALL_10-800_LOOSE" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
region_cat_fname = region_fname.replace(".fits","_cat.fits")

do_cat = False                        # Recreate catalog file from vasca_pipe output region
do_cross_match = False                  # Does source matching with CDS
do_src_prep = True                  # Gets additional info for sources listes in srcs_ids list below.
do_variable_addition = False         # Adds variables of matched sources to tt_sources
do_LombScargle = False

srcs_ids = []
srcs_ids = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,253670
            ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,645215 ,645730
            ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206 ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astropy import units as uu
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
logger.enable("vasca")

# Do catalog file

if do_cat:
    rg = Region()
    rg.load_from_fits(region_fname) 
    rc = rg.get_region_catalog()
else:
    rc = Region()
    rc.load_from_fits(region_cat_fname)

#Cross match to SIMBAD and GAIA
if do_cross_match:
    rc.cross_match_cds(query_table="simbad", overwrite=False)
    rc.cross_match_cds(overwrite=True)
    #rc.cross_match_cds(query_table="I/358/varisum",vizier_columns=["*", "NG", "dGmag"],)

#Add variables
if do_variable_addition:
    rc.add_column("tt_gaiadr3", "Gmag_abs")
    sel_plx = rc.tt_gaiadr3["RPlx"]>3
    rc.tt_gaiadr3["Gmag_abs"][sel_plx]=rc.tt_gaiadr3["Gmag"][sel_plx] + 5.0 * np.log10(rc.tt_gaiadr3["Plx"][sel_plx]) - 10.0
    rc.tt_gaiadr3["Plx_dist"] = rc.tt_gaiadr3["Plx"].quantity.to(uu.parsec, equivalencies=uu.parallax())

    # Add normalized excess
    rc.tt_sources["flux_ne"]=np.sqrt(rc.tt_sources["flux_nxv"])

    # Add object groups
    add_ogrp(rc.tt_simbad)
    add_ogrp(rc.tt_gaiadr3,provenance="GAIA")

if do_LombScargle:
    rc.set_LombScargle(nbins_min=20)

#Write out all changes to region
rc.write_to_fits(region_cat_fname, overwrite=True)

#Prepare sources, including Vizier SED.
if do_src_prep:
    for src_id in srcs_ids:
        print(f"*********** Creating Source Table ID {src_id} *****************")
        tc_src = rc.get_src_from_id(src_id, load_from_file = False)
print("Done")

2023-11-02 15:02:49.893 | DEBUG    | vasca.tables:load_from_fits:330 - Loading file with name './vasca_pipeline/ALL_10-800_LOOSE/region_ALL_10-800_LOOSE_cat.fits'
2023-11-02 15:02:49.902 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_fields'
2023-11-02 15:02:49.916 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_filters'
2023-11-02 15:02:49.922 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_visits'
2023-11-02 15:02:49.930 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_sources'
2023-11-02 15:02:49.956 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_detections'
2023-11-02 15:02:49.995 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_detections'
2023-11-02 15:02:50.082 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_coadd_sources'
2023-11-02 15:02:50.102 | DEBUG    | vasca.tables:load_from_fits:341 - Loading table 'tt_src_id_map'
2023-11-02 15:02:50.111 | DEBUG   

*********** Creating Source Table ID 4993 *****************


2023-11-02 15:02:52.152 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-02 15:02:52.161 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.161 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.161 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.162 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.162 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.162 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.163 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.163 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:02:52.163 | WARNING  | vasca.source:add_vizier_SED:99 - Ski

*********** Creating Source Table ID 13580 *****************


2023-11-02 15:03:03.308 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-02 15:03:03.310 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.328 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.329 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.329 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.330 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.330 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.330 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.356 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:03:03.356 | WARNING  | vasca.source:add_vizier_SED:99 - Ski

...

*********** Creating Source Table ID 2527679 *****************


2023-11-02 15:10:50.002 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-02 15:10:50.006 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:10:50.008 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:10:50.030 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_gphoton_lc'
2023-11-02 15:10:50.034 | WARNING  | vasca.source:add_spectrum:276 - No spectrum found
2023-11-02 15:10:50.036 | INFO     | vasca.tables:write_to_fits:224 - Writing file with name './vasca_pipeline/ALL_10-800_LOOSE/sources/src_2527679.fits'
2023-11-02 15:10:50.039 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sources'
2023-11-02 15:10:50.076 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_detections'
2023-11-02 15:10:50.089 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_gaiadr3'
2023-11-02 15:10:50.262 | DEBUG    | vasca.tables:write_to_f

*********** Creating Source Table ID 3074917 *****************


2023-11-02 15:11:06.240 | DEBUG    | vasca.tables:add_table:159 - Adding table 'region:tt_sed'
2023-11-02 15:11:06.242 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:11:06.261 | WARNING  | vasca.source:add_vizier_SED:99 - Skipping row as flux contains nan
2023-11-02 15:11:06.265 | WARNING  | vasca.source:add_gphoton_lc:206 - gPhoton file not found /home/buehler/Data/galex/GALEX_gPhoton/gPhoton_ra10.881_dec-9.213_nuv_app.npy
2023-11-02 15:11:06.269 | WARNING  | vasca.source:add_spectrum:276 - No spectrum found
2023-11-02 15:11:06.270 | INFO     | vasca.tables:write_to_fits:224 - Writing file with name './vasca_pipeline/ALL_10-800_LOOSE/sources/src_3074917.fits'
2023-11-02 15:11:06.272 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_sources'
2023-11-02 15:11:06.312 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt_detections'
2023-11-02 15:11:06.326 | DEBUG    | vasca.tables:write_to_fits:264 - Writing table 'tt

Done
